## 텐서플로 import

In [10]:
!pip install tensorflow-gpu

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\ml\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\user\anaconda3\envs\ml\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\user\anaconda3\envs\ml\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\user\anaconda3\envs\ml\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\user\anaconda3\envs\ml\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\user\anaconda3\envs\ml\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\user\anaconda3\envs\ml\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\user\anaconda3\envs\ml\lib\ssl.py", line 109

In [11]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

## MNIST 데이터셋 다운로드 하고 준비하기

In [13]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data() # 데이터 셋 받아오기

train_images = train_images.reshape((60000, 28, 28, 1)) # 60000개, 28x28, 1은 채널
test_images = test_images.reshape((10000, 28, 28, 1))

train_images, test_images = train_images / 255.0, test_images / 255.0 # 흑백이미지는 0~255사이의 값을 가지기 때문에 train_images, test_images가 0~1 사이의 값을 갖도록 정규화

11490434/11490434 [==============================] - 5s 0us/step


## 합성곱 층 만들기

#### Conv2D와 MaxPooling2D 층을 쌓는 일반적인 패턴으로 합성곱 층을 정의
- 첫 번째 층의 매개변수 input_shape으로 (28,28,1) 전달
- (이미지 높이, 이미지 너비, 컬러채널)
- 흑백 이미지 -> 컬러 채널=1 / 컬러 이미지 -> 컬러채널=3 (R,G,B)

In [18]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3, 3),activation='relu',input_shape=(28, 28, 1))) # 필터개수,(필터크기)=32개,3x3크기
model.add(layers.MaxPooling2D((2,2))) # pool size=2x2 -> MaxPooling:최댓값, strides를 지정하지 않으면 pooling 필터의 크기와 같아짐 -> 2칸씩 이동
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

#### 모델의 구조

- Conv2D와 MaxPooling2D 층의 출력(Output Shape)은 (높이, 너비, 채널) 크기의 3차원 텐서
- 네트워크가 깊어질 수록 Output Shape이 점점 줄어드는 것을 볼 수 있음

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trai

## 마지막에 Dense 층 추가하기

Dense 층은 벡터(1차원)를 입력받기 때문에, 마지막 합성곱 층의 출력 텐서(3차원)를 1차원으로 펼쳐야 한다.

- relu: 입력이 양수일 때는 그대로 출력, 음수일때는 0 출력
- softmax: 입력받은 값을 0~1사이의 출력이 되도록 정규화하여 출력 값들의 총합이 항상 1이 되도록하는 특성을 가진 함수

In [21]:
model.add(layers.Flatten()) # 1차원 배열로 만들기 -> 576개
model.add(layers.Dense(64,activation='relu')) # 출력층 노드 개수 64개
model.add(layers.Dense(10,activation='softmax')) # 출력층 노드 개수 10개(0~9의 숫자), 분류를 여러가지로 할 때 softmax 사용

#### 최종 모델의 구조

In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)              

## 모델 컴파일과 훈련

훈련을 진행할 수록 정확도는 점점 증가하며, 손실은 감소한다.

In [27]:
model.compile(optimizer='adam', # 최적화
              loss='sparse_categorical_crossentropy', # 손실함수:실제값과 예측값의 차이를 수치화
              metrics=['accuracy']) # 기준

model.fit(train_images, train_labels, epochs=5) # 5회 훈련

Epoch 1/5
1875/1875 [==============================] - 48s 25ms/step - loss: 0.1451 - accuracy: 0.9558
Epoch 2/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0467 - accuracy: 0.9852
Epoch 3/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0336 - accuracy: 0.9894
Epoch 4/5
1875/1875 [==============================] - 46s 25ms/step - loss: 0.0258 - accuracy: 0.9920
Epoch 5/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0202 - accuracy: 0.9937


## 모델 평가

In [28]:
test_loss, test_acc=model.evaluate(test_images,test_labels,verbose=2)

313/313 - 4s - loss: 0.0240 - accuracy: 0.9918 - 4s/epoch - 12ms/step


손실 함수 값이 0.0240, 정확도 0.9918이다.

In [29]:
print(test_acc)

0.9918000102043152


간단한 CNN 모델이 99%의 테스트 정확도를 달성함을 볼 수 있다.